In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('C:\\Users\\anjah\\Documents\\faks\\magisterij\\1. Letnik\\strojno učenje\\seminarska\\raw_data.csv', sep = ',')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['Health Indicator']=df['Health Indicator'].fillna(df['Health Indicator'].mode()[0])

In [ ]:
df['Holding_Policy_Type'] = df['Holding_Policy_Type'].fillna(df['Holding_Policy_Type'].mode()[0])
df['Holding_Policy_Duration'] = df['Holding_Policy_Duration'].fillna(df['Holding_Policy_Duration'].mode()[0])

In [ ]:
correlation = df.corr().round(2)
plt.figure(figsize = (14,7))
comap = sns.color_palette("Spectral", as_cmap=True)
mask = np.triu(np.ones_like(df.corr()))
dataplot = sns.heatmap(df.corr(), annot = True, cmap = comap, mask = mask)
plt.savefig('C:\\Users\\anjah\\Documents\\faks\\magisterij\\1. Letnik\\strojno učenje\\seminarska\\corelations.png',format = 'png', bbox_inches = 'tight', bpi = 300)

In [ ]:
ne, ja = df['Response'].value_counts()
fig = plt.figure(figsize=(7,5))
plt.bar(0, ne, color = 'teal', width = 0.5)
plt.title('Response')
plt.bar(1, ja, color = 'slateblue', width = 0.5)
plt.xticks([0,1])
plt.savefig('C:\\Users\\anjah\\Documents\\faks\\magisterij\\1. Letnik\\strojno učenje\\seminarska\\response.png',format = 'png', bbox_inches = 'tight', bpi = 300)

In [ ]:
plt.figure(figsize = (15,15))



plt.subplot(3,1,1)
plt.title("City_Code")
sns.countplot(x = 'City_Code', hue= 'Response', palette=['teal','slateblue'], data = df)

plt.subplot(3,1,2)
sns.countplot(x = 'Health Indicator', hue= 'Response', palette=['teal','slateblue'], data = df)
plt.title("Health Indicator")

plt.subplot(3,1,3)
sns.countplot(x = 'Holding_Policy_Duration', hue= 'Response', palette=['teal','slateblue'], data = df)
plt.title("Holding_Policy_Duration")
plt.savefig('C:\\Users\\anjah\\Documents\\faks\\magisterij\\1. Letnik\\strojno učenje\\seminarska\\comp_cc_hi_hpd.png',format = 'png', bbox_inches = 'tight', dpi = 300)

In [ ]:
plt.figure(figsize = (20,15))

plt.subplot(2,3,1)
sns.countplot(x = 'Accomodation_Type', hue= 'Response', palette=['teal','slateblue'], data = df)
plt.title("Accomodation_Type")

plt.subplot(2,3,2)
sns.countplot(x = 'Reco_Insurance_Type', hue= 'Response', palette=['teal','slateblue'], data = df)
plt.title('Reco_Insurance_Type')

plt.subplot(2,3,3)
sns.countplot(x = 'Is_Spouse', hue= 'Response', palette=['teal','slateblue'], data = df)
plt.title('Is_Spouse')
"""
plt.subplot(2,1,2)
sns.countplot(x = 'Holding_Policy_Type', hue= 'Response', palette='hls', data = df)
"""
plt.savefig('C:\\Users\\anjah\\Documents\\faks\\magisterij\\1. Letnik\\strojno učenje\\seminarska\\at_rit_is.png',format = 'png', bbox_inches = 'tight', dpi = 300)

For prediction drop: ID, Holding_Policy_Duration, Holding_Policy_Type

get_dummies --> pretvoriš catgerocial tako da dodaš stolpce in če je nek city code v vrstici je tam 1. Uporabiš za vse categorical!!!!!

dodej data da bo bl enakomerna da in ne?

In [ ]:
df = df.drop('ID', 1)

In [ ]:
df['Is_Spouse'].replace(('Yes','No'), (1,0), inplace = True)
df['Reco_Insurance_Type'].replace(('Individual', 'Joint'), (1,0), inplace = True)
df['Accomodation_Type'].replace(('Owned','Rented'), (0,1), inplace = True)

In [ ]:
df['Health Indicator'].replace(('X1','X2','X3','X4','X5','X6','X7','X8','X9'), (0,1,2,3,4,5,6,7,8), inplace = True)

In [ ]:
df['Holding_Policy_Duration'].replace(('14+'), (14), inplace = True)

In [ ]:
df

In [ ]:
city_code = pd.get_dummies(df['City_Code'])
df = pd.concat([df, city_code], axis = 1)
df = df.drop('City_Code', axis = 1)

In [ ]:
response = (df['Response'])
df = pd.concat([df, response], axis = 1)
df = df.drop('Response', axis = 1)

In [ ]:
df['response'] = response

In [ ]:
#seperate into feature and target variables
X = df.iloc[:,0:47].values
Y = df.iloc[:, 47].values

In [ ]:
cnum = len(df.columns)
cnum

v tem primeru je random over sampler
lahko se drugi nacini over samplanja???

In [ ]:
# balancing classes (yes, no (1,0))
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X, Y = ros.fit_resample(X, Y)
ax = sns.countplot(x=Y)


najprej brez scalanja
lahko probas se standard scaling in min max scaling

Transforming Data into Train and Test, here we will use 30% of our data to test the machine learning models

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

DECISION TREE

In [ ]:
min_split = np.array([2, 3, 4, 5, 6, 7])
max_nvl = np.array(range(0,48))
alg = ['entropy', 'gini']
values_grid = {'min_samples_split': min_split, 'max_depth': max_nvl, 'criterion': alg}

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
model = DecisionTreeClassifier()
gridDecisionTree = GridSearchCV(estimator = model, param_grid = values_grid, cv = 5)

tuki finetunamo hyperparameters

In [ ]:
gridDecisionTree.fit(X_train, Y_train)

In [ ]:
print('Mín Split: ', gridDecisionTree.best_estimator_.min_samples_split)
print('Max Nvl: ', gridDecisionTree.best_estimator_.max_depth)
print('Algorithm: ', gridDecisionTree.best_estimator_.criterion)
print('Score: ', gridDecisionTree.best_score_)

RUNNING DECISION TREE

In [ ]:
decision_tree = DecisionTreeClassifier(criterion = 'gini', min_samples_split = 3, max_depth= 100, random_state=0)

In [ ]:
decision_tree = DecisionTreeClassifier()

In [ ]:
decision_tree.fit(X_train, Y_train)

In [ ]:
pred_tests = decision_tree.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisp
cm = confusion_matrix(Y_test,pred_tests, labels = decision_tree.classes_)
vis = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=decision_tree.classes_)
vis.plot(cmap = 'viridis')
vis.ax_.set_title("Decision tree")
vis.ax_.grid()
vis.figure_.savefig('Dtree_cm.png', dpi = 300)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
classification_decision = (classification_report(Y_test, pred_tests))
print(classification_decision)

In [ ]:
dtree_CA = 0.83

Preverimo kateri attributi najbolj vplivajo --> ignoriramo ostale mogoče bo boljši score. OCENA ATRIBUTOV ZA DECISION TREE

VZAMIMO SAMO NAJPOMEMBNEJŠE FEATURSE: MEJA JE 0.03    NE POMAGAAA :(  CA = 0.618

RANDOM FOREST !!!!!!!!!!!

In [ ]:
from sklearn.ensemble import RandomForestClassifier
n_estimators = np.array([100])
est = ['entropy', 'gini']
values_grid = {'n_estimators': n_estimators, 'criterion': est}

In [ ]:
model = RandomForestClassifier()
gridRandomForest = GridSearchCV(estimator = model, param_grid = values_grid, cv = 10)
gridRandomForest.fit(X_train, Y_train)

In [ ]:
print('Algorithm: ', gridRandomForest.best_estimator_.criterion)
print('Score: ', gridRandomForest.best_score_)

In [ ]:
gridRandomForest.best_params_


spet izberemo gini

In [ ]:
random_forest = RandomForestClassifier()
random_forest.fit(X_train, Y_train)

In [ ]:
pred_tests = random_forest.predict(X_test)

In [ ]:
cm_rf = confusion_matrix(Y_test,pred_tests, labels = decision_tree.classes_)
disp_rf = ConfusionMatrixDisplay(confusion_matrix=cm_rf,
                              display_labels=decision_tree.classes_)
disp_rf.plot(cmap = 'viridis')
disp_rf.ax_.set_title("Random forrest")
disp_rf.ax_.grid()
disp_rf.figure_.savefig('rforrest_cm.png', dpi = 300)

In [ ]:
classification_random = (classification_report(Y_test, pred_tests))
print(classification_random)


In [ ]:
rforrest_CA = 0.895

LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(random_state =45 )
logistic.fit(X_train, Y_train)

In [ ]:
pred_tests = logistic.predict(X_test)

In [ ]:
cm_lr = confusion_matrix(Y_test,pred_tests, labels = logistic.classes_)
disp_lr = ConfusionMatrixDisplay(confusion_matrix=cm_lr,
                              display_labels=logistic.classes_)
disp_lr.plot(cmap = 'viridis')
disp_lr.ax_.set_title("Logistic regression")
disp_lr.ax_.grid()
disp_lr.figure_.savefig('logreg_cm.png', dpi = 300)

In [ ]:
logistic_normal = (classification_report(Y_test, pred_tests))
print(logistic_normal)

In [ ]:
logreg_CA = 0.545

NAIVE BAYES

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.naive_bayes import GaussianNB
nb_classifier = GaussianNB()

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
gs_NB = GridSearchCV(estimator=nb_classifier,
                 param_grid=params_NB,
                 cv=RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1),   # use any cross validation technique
                 verbose=1,
                 scoring='accuracy')
gs_NB.fit(X_train, Y_train)

gs_NB.best_params_

In [ ]:
from sklearn.naive_bayes import GaussianNB
naive_bayes = GaussianNB(var_smoothing= 2.310129700083158e-09)
naive_bayes.fit(X_train, Y_train)

In [ ]:
pred_tests_nb = naive_bayes.predict(X_test)

In [ ]:
cm_nb = confusion_matrix(Y_test,pred_tests_nb, labels = naive_bayes.classes_)
disp_nb = ConfusionMatrixDisplay(confusion_matrix=cm_nb,
                              display_labels=naive_bayes.classes_)
disp_nb.plot(cmap = 'viridis')
disp_nb.ax_.set_title("Naive Bayes")
disp_nb.ax_.grid()
disp_nb.figure_.savefig('naiveB_cm.png', dpi = 300)

In [ ]:
nBayes_CA = 0.552

K SOSEDI

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
knn = KNeighborsClassifier()

In [ ]:
k = list(range(1,10))
k_values = dict(n_neighbors = k_list)
grid = GridSearchCV(knn, k_values, cv = 10, scoring = 'accuracy')
grid.fit(X_train, Y_train)

In [ ]:
grid.param_grid, grid.best_score_

In [ ]:
grid.best_params_, grid.best_score_

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 1, metric = 'minkowski', p = 2)
knn.fit(X_train, Y_train)

In [ ]:
pred_tests_k = knn.predict(X_test)

In [ ]:
cm_knn = confusion_matrix(Y_test,pred_tests_k, labels = knn.classes_)
disp_knn = ConfusionMatrixDisplay(confusion_matrix=cm_knn,
                              display_labels=knn.classes_)
disp_knn.plot(cmap = 'viridis')
disp_knn.ax_.set_title("K nearest neibghbors")
disp_knn.ax_.grid()
disp_knn.figure_.savefig('knn_cm.png')

In [ ]:
knn_CA = 0.814

COMBINE MACHINE LEARNING MODELS - VOTING

+ bayes : ca = 0.789    bed
+ bayes + knn: ca = 0.837
dejmo zbrisat bayesa
+ brez bayesa ca=0.834

S cross validation!

In [ ]:

# compare standalone models for binary classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from matplotlib import pyplot


# get a list of models to evaluate
def get_models():
    models = dict()
    models['lr'] = LogisticRegression()
    models['knn'] = KNeighborsClassifier(n_neighbors = 1, metric = 'minkowski', p = 2)
    models['cart'] = DecisionTreeClassifier()
    models['rf'] = RandomForestClassifier()
    models['bayes'] = GaussianNB()
    return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, Y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()
pyplot.savefig('')

In [ ]:

from numpy import mean
from numpy import std
from matplotlib import pyplot

# get the dataset

def get_stacking():
 # define the base models
     level0 = list()
     level0.append(('lr', LogisticRegression()))
     level0.append(('knn', KNeighborsClassifier(n_neighbors = 1, metric = 'minkowski', p = 2)))
     level0.append(('dt', DecisionTreeClassifier()))
     level0.append(('rf', RandomForestClassifier()))
     level0.append(('bayes', GaussianNB(var_smoothing= 2.310129700083158e-09)))
     # define meta learner model
     level1 = GaussianNB()
     # define the stacking ensemble
     model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
     return model
# get a list of models to evaluate
def get_models():
    models = dict()
    models['lr'] = LogisticRegression()
    models['knn'] = KNeighborsClassifier(n_neighbors = 1, metric = 'minkowski', p = 2)
    models['dt'] = DecisionTreeClassifier()
    models['rf'] = RandomForestClassifier()
    models['bayes'] = GaussianNB(var_smoothing= 2.310129700083158e-09)
    models['stacking'] = get_stacking()
    return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

# define dataset

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, Y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

In [ ]:
from numpy import mean
from numpy import std

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot

# get the dataset

def get_stacking():
 # define the base models
     level0 = list()
     level0.append(('lr', LogisticRegression()))
     level0.append(('knn', KNeighborsClassifier(n_neighbors = 1, metric = 'minkowski', p = 2)))
     level0.append(('dt', DecisionTreeClassifier()))
     level0.append(('rf', RandomForestClassifier()))
     level0.append(('bayes', GaussianNB(var_smoothing= 2.310129700083158e-09)))
     # define meta learner model
     level1 = GaussianNB()
     # define the stacking ensemble
     model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
     return model
# get a list of models to evaluate
def get_models():
    models = dict()
    #models['lr'] = LogisticRegression()
    #models['knn'] = KNeighborsClassifier(n_neighbors = 1, metric = 'minkowski', p = 2)
    #models['dt'] = DecisionTreeClassifier()
    #models['rf'] = RandomForestClassifier()
    #models['bayes'] = GaussianNB(var_smoothing= 2.310129700083158e-09)
    models['stacking'] = get_stacking()
    return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores

# define dataset

# get the models to evaluate
models_stac = get_models()
# evaluate the models and store results
results_stac, names_stac = list(), list()
for name, model in models_stac.items():
    scores_stac = evaluate_model(model, X, Y)
    results_stac.append(scores_stac)
    names_stac.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results_stac, labels=names, showmeans=True)
pyplot.show()

In [ ]:
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.title('Comparison of models')
pyplot.ylabel('Classification accuracy')
pyplot.xticks([1,2,3,4,5,6],['LRM', 'KNN', 'DT','RFC', 'NB', 'stacking'])
pyplot.savefig('comparison of models', dpi=300)
pyplot.show()


In [ ]:
results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

data = np.linspace(-20, 20, 200)
sm = 1/(1 + np.exp(-data))
x1, y1 = [-30, 30], [1, 1]
x2, y2 = [-30,30],[0.5,0.5]
fig = plt.figure(figsize=(5,4))
plt.plot(data, sm, color = 'slateblue')
plt.xlabel("X")
plt.ylabel("Probability of the event \n occurring", fontsize = 13)
plt.xticks(ticks = [], labels = [])
plt.yticks([0,1],[0,1])
plt.plot(x1,y1, linestyle='dotted', color = 'coral')
#plt.title('Logistic regression')
plt.plot([-12,-4,-7,-2,-3,2,3,6,13,7,-13, -5, -9, 11, 2.5,5],[0.005,0.01,0.02,0.1,0.13,0.8,0.94,0.95,0.98,0.99, 0.003, 0.025, 0.002,0.989,0.9,0.97], lw = 0, ms = 10, marker = 'o', alpha = 0.5)
plt.plot(x2,y2, linestyle = 'dashed', color = 'forestgreen', lw = 1)

fig.text(0.75,0.52,'threshold', fontsize = 12)

plt.xlim(-20,20)
plt.grid()
plt.savefig('logistic_regression', dpi = 300)

plt.show()

In [ ]:
models